getting  dataset

In [3]:
import tensorflow as tf
import urllib.request
import os 
import zipfile

2023-11-30 09:24:53.950795: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 09:24:54.025044: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 09:24:54.025089: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 09:24:54.025139: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 09:24:54.040335: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 09:24:54.041027: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [1]:

url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"

tmpDirectory = os.getcwd() +"/tmp"

if not os.path.exists(tmpDirectory):
    os.makedirs(tmpDirectory)

filename = tmpDirectory+"/cats_and_dogs_filtered.zip"
urllib.request.urlretrieve(url,filename)

zipRef = zipfile.ZipFile(filename,"r")
zipRef.extractall(tmpDirectory)
zipRef.close()


In [4]:
baseDir = os.path.join(tmpDirectory,"cats_and_dogs_filtered")

trainDir = os.path.join(baseDir,"train")
validationDir = os.path.join(baseDir,"validation")

trainCatsDir = os.path.join(trainDir,"cats")
trainDogsDir = os.path.join(trainDir,"dogs")

validationCatsDir = os.path.join(validationDir,"cats")
validationDogsDir = os.path.join(validationDir,"dogs")

trainCatNames = os.listdir(trainCatsDir)
trainDogNames = os.listdir(trainDogsDir)

print(trainCatNames[:10])
print(trainDogNames[:10])


['cat.967.jpg', 'cat.42.jpg', 'cat.81.jpg', 'cat.983.jpg', 'cat.601.jpg', 'cat.358.jpg', 'cat.109.jpg', 'cat.637.jpg', 'cat.427.jpg', 'cat.818.jpg']
['dog.330.jpg', 'dog.261.jpg', 'dog.253.jpg', 'dog.404.jpg', 'dog.960.jpg', 'dog.758.jpg', 'dog.904.jpg', 'dog.146.jpg', 'dog.414.jpg', 'dog.236.jpg']


In [5]:

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(16,(3,3),activation="relu", input_shape=(150,150,3) ),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32,(3,3),activation="relu", input_shape=(150,150,3) ),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3),activation="relu", input_shape=(150,150,3) ),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512,activation="relu"),
        tf.keras.layers.Dense(1,activation="sigmoid")
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 64)        0

In [15]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])

In [16]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.)
train_generation = train_datagen.flow_from_directory(trainDir,batch_size=20,class_mode="binary",target_size=(150,150))
validation_generation = test_datagen.flow_from_directory(validationDir,batch_size=20,class_mode="binary",target_size=(150,150))

Found 2000 images belonging to 2 classes.


Found 1000 images belonging to 2 classes.


In [17]:
history = model.fit(train_generation,validation_data=validation_generation,steps_per_epoch=100,epochs=15,validation_steps=50,verbose=2)

Epoch 1/15
100/100 - 38s - loss: 0.7289 - accuracy: 0.5400 - val_loss: 0.6413 - val_accuracy: 0.6030 - 38s/epoch - 382ms/step
Epoch 2/15
100/100 - 40s - loss: 0.6252 - accuracy: 0.6455 - val_loss: 0.5974 - val_accuracy: 0.6750 - 40s/epoch - 405ms/step
Epoch 3/15
100/100 - 44s - loss: 0.5879 - accuracy: 0.6900 - val_loss: 0.5791 - val_accuracy: 0.6840 - 44s/epoch - 437ms/step
Epoch 4/15
100/100 - 41s - loss: 0.5523 - accuracy: 0.7105 - val_loss: 0.5712 - val_accuracy: 0.7060 - 41s/epoch - 406ms/step
Epoch 5/15
100/100 - 41s - loss: 0.5025 - accuracy: 0.7600 - val_loss: 0.6008 - val_accuracy: 0.6890 - 41s/epoch - 409ms/step
Epoch 6/15
100/100 - 40s - loss: 0.4467 - accuracy: 0.7805 - val_loss: 0.6059 - val_accuracy: 0.7150 - 40s/epoch - 398ms/step
Epoch 7/15
100/100 - 43s - loss: 0.4048 - accuracy: 0.8130 - val_loss: 0.5932 - val_accuracy: 0.7100 - 43s/epoch - 432ms/step
Epoch 8/15
100/100 - 42s - loss: 0.3461 - accuracy: 0.8455 - val_loss: 0.6799 - val_accuracy: 0.7100 - 42s/epoch - 416